In [21]:

import os
import sys
parent = os.path.realpath(os.path.join(os.path.abspath(''), os.pardir))
sys.path.append(parent)
import numpy as np
from databases.sql_connect import create_connection
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import copy
from scipy.stats import lognorm
from statistics import median, mean

cats = {
    "EDIP-Gold": "Gold [kg]",
    "EDIP-Copper": "Copper [kg]",
    #"ADP-Minerals- Total": "Depletion of Minerals [kg Sb-eq]",
    "EI-Minerals- Total": "Economic Importance [kg Sb-eq]",
    #"EI-Minerals-Precious Metals": "Precious Metals",
    "GPR-Minerals- Total": "Geo-Politic Supply Risk [1]",
    #"CML-climate change (GWP 100a)": "Green House Emissions",
    "CED - total": "Primary Energy [MJ-eq]",
    "EcoIndicator 99 (E - E) - total":"Total Impacts",
    #"ReCiPe Endpoint (E - A) - total (total)": "Total Impacts",
}


sns.set_theme()
sns.set(context="paper", style="darkgrid", font="Times New Roman", font_scale = 2)
cm = 1/2.54
FONTSIZE = 8

def get_label(shape,scale):
    sigma = "\u03C3"
    mu ="\u03BC"
    geomUnc= np.exp(shape)
    scale_val = "{:.2E}".format(scale)
    geomUnc = "{:.4}".format(geomUnc)
    label = f"{mu} = {scale_val}, " + f"{sigma} = {geomUnc}"
    return label

In [22]:

# "GPR-Minerals- Total" #"ADP-Minerals-Copper" #"ReCiPe Midpoint (E) - metal depletion (MDP)" #"CED - total" #"EDIP-Gold" # "GPR-Minerals- Total"
s_x = 15
s_y = 15
def analyse_montecarlo(ps_name):
    base= "data/MC_TEMPRO_DB220425_{}_{}.csv".format("Base", ps_name)
    corr = "data/MC_TEMPRO_DB220425_{}_{}.csv".format("Corr", ps_name)

    df_mc_base=pd.read_csv(base)
    df_mc_corr=pd.read_csv(corr)
    fig, axs = plt.subplots(3,2,figsize=(s_x,s_y))
    ax_=[axs[i][j] for i in range(3) for j in range(2)]
    i=0
    for col in cats.keys():
        data = df_mc_base[col]
        data2 = df_mc_corr[col]
        rows= min(len(data),len(data2))
        data=data[:rows]
        data2=data2[:rows]
        d = {"Base":data, "Corr":data2}
        df = pd.DataFrame(d)
        df=df[df["Base"]>0]
        df=df[df["Corr"]>0]
        shape, loc, scale = stats.lognorm.fit(df["Base"], floc=0)
        shape2, loc2, scale2 = stats.lognorm.fit(df["Corr"], floc=0)

        #df=df[df["Base"]<4*scale]
        #df=df[df["Corr"]<4*scale2]

        #'shape, loc, scale = stats.lognorm.fit(df["Base"], floc=0)
        # shape2, loc2, scale2 = stats.lognorm.fit(df["Corr"], floc=0)

        def get_label(shape,scale):
            sigma = "\u03C3"
            mu ="\u03BC"
            geomUnc= np.exp(shape)
            scale_val = "{:.2e}".format(scale)
            geomUnc = "{:.4}".format(geomUnc)
            label = f"{mu} = {scale_val}\n" + f"{sigma} = {geomUnc}"
            return label

        label0 = get_label(shape,scale)
        label2 = get_label(shape2,scale2)
        df_norm= copy.deepcopy(df)
        scale_median_base = mean(df_norm["Base"])
        scale_median_corr = mean(df_norm["Corr"])

        df_norm["Base"]*=(1/scale)
        df_norm["Corr"]*=(1/scale2)
        #, palette="colorblind",
        ax = ax_[i]
        sns.violinplot(data=df_norm, ax=ax,orient ='v',palette="viridis")

        ax.set_title(cats[col],fontweight='bold')

        #ax.title()
        ax.set_ylim([0, 3])
        #if col =="GPR-Minerals- Total":
            #ax.set_yscale("log")
        
        ax.set_xticklabels([label0,label2])#labels, *, fontdict=None, minor=False, **kwargs)
        i+=1

    from matplotlib.lines import Line2D
    # https://imagecolorpicker.com/en
    c1= "#3c6682"
    c2 ="#45a778"
    custom_lines = [Line2D([0], [0], color=c1, lw=8, label="ecoinvent 3.4"),
                    Line2D([0], [0], color=c2, lw=8, label="Updated Values"),
                    ]
    
    fig.legend(bbox_to_anchor=(0.5,-0.03),handles=custom_lines,ncols=2, loc = "lower center")
    fig.tight_layout(h_pad=1, w_pad=1)
    #fig.suptitle("Comparison of Result Uncertainty Values (after laboratory measurements) \n" + ps_name,
    #             fontsize=16,fontweight='bold',y=1.075)

    fig.savefig(f"Violin/MC-Analysis{ps_name}.png",bbox_inches='tight',dpi=900)
    plt.close()
#analyse_montecarlo("3204-CPU, mounted mainboard")


In [23]:
ps_list = ["3204-CPU, mounted mainboard",
"3208-Mainboard",
"3211-PCB mounted mainboard",
"3219-PCB, for power supply unit",
"4440-Server, 1U, no storage",
"4443-Server, 2U, storage",
"4455-Blade (Ref. Unit)",
"5101-KDO-IT",
"5201-KDO-Climatization",
"5301-KDO-Energy Supply",
"5401-KDO-Infrastructure",
"6001-KDO",
"6003-RZ1"]

for ps in ps_list[0:1]:
    analyse_montecarlo(ps)
    pass
